# folium 시각화
참고할 만한 링크

- folium 공식 documentation(eng) : https://python-visualization.github.io/folium/
- folium 패키지 살펴보기(kor) :  https://dailyheumsi.tistory.com/85
- choropleth(eng) :    https://vverde.github.io/blob/interactivechoropleth.html

### 패키지/모듈 임포트

In [1]:
import pandas as pd
import folium
import branca.colormap as cm
import json
import matplotlib.pyplot as plt

pd.options.display.max_rows = 150
pd.options.display.max_columns = 350
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

path1 = './data/'
path2 = './map/' 

### 파일 임포트

'map'이라는 하위 폴더 만들어 아래 링크에 있는 파일 4개 넣어두기

http://bit.ly/2O83K2F
- 행정구역코드.csv
- sigungu.shp
- sido.shp
- east.png

In [2]:
sido_data=pd.read_csv(path1+'imp_시도별지역안전지표_2015.csv', encoding='cp949')
sigungu_data=pd.read_csv(path1+'imp_시군구별지역안전지표_2015.csv', encoding='cp949')

#map폴더 내에 있어야 하는 파일
code=pd.read_csv(path2+'행정구역코드.csv', encoding='cp949')
sido_shp=json.load(open(path2+'sido.json', encoding='utf=8'))
sigungu_shp=json.load(open(path2+'sigungu.json', encoding='utf=8'))

### 기본 지도 생성해보기

In [3]:
#기본지도생성
m = folium.Map(location=[36, 127],tiles=None, min_zoom=4, zoom_start=6.2)

#지도타일 추가
folium.TileLayer('cartodbpositron',control=True, min_zoom=5, zoom_start=6.2).add_to(m)

#동해 패치
icon = folium.features.CustomIcon(path2+'east.png',icon_size=(120,90))
folium.Marker(location=[40.2747,134.85077],icon=icon).add_to(m)

m

### choropleth_1개 열

In [5]:
#기본지도생성
m = folium.Map(location=[36, 127],tiles=None, min_zoom=4, zoom_start=6.2)
folium.TileLayer('cartodbpositron',control=True, min_zoom=5, zoom_start=6.2).add_to(m)
icon = folium.features.CustomIcon(path2+'east.png',icon_size=(120,90))
folium.Marker(location=[40.2747,134.85077],icon=icon).add_to(m)

col='1인가구수' #''안에 확인하고 싶은 컬럼명 넣기

#colorscale나누기, 지역안전지수 상대등급 구간 10:25:30:25:10으로 범위 나눔
bins = list(sigungu_data[col].quantile([0, 0.1, 0.35, 0.65, 0.9, 1]))

folium.Choropleth(
    geo_data=sigungu_shp, #지역 폴리곤(시도(sido_shp),시군구(sigungu_shp))
    name=col,
    data=sigungu_data, #통계데이터(시도(sido_data),시군구(sigungu_data))
    columns=['시군구',col], #시도/시군구
    key_on='feature.properties.'+'SIG_KOR_NM', #시도(CTP_KOR_NM),시군구(SIG_KOR_NM)
    line_weight=0.1,
    fill_color='RdBu_r',#높은수치가 안전하면 RdBu, 낮은수치가 안전하면 RdBu_r, 
    fill_opacity=0.8,
    nan_fill_opacity=0,
    overlay=True,
    bins=bins,
    legend_name=col,
    highlight=True,
    reset=True
).add_to(m)

#tooltip만들기(마우스 올리면 위치에 따른 정보 보이는 것)
style_function = lambda x: {'fillColor': '#ffffff', #말풍선색(흰색)
                            'color':'#000000', #글자색(검정)
                            'fillOpacity': 0.1, 
                            'weight': 0.1}

ttip = folium.features.GeoJson(
    sigungu_shp,
    style_function=style_function, 
    control=False, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['SIG_KOR_NM'], #시도(CTP_KOR_NM),시군구(SIG_KOR_NM)
        aliases=['시군구명'] #시도(시도명),시군구(시군구명)
    )
)

ttip.add_to(m)

m.keep_in_front(ttip)

folium.LayerControl(collapsed=True).add_to(m)

m
#지도가 안 보이면 아랫줄 주석 풀어 html로 저장해서 html문서 열어봐도 된다
#m.save('choropleth.html')

### choropleth_다수열

In [7]:
#기본지도생성
m = folium.Map(location=[36, 127],tiles=None, min_zoom=4, zoom_start=6.2)
folium.TileLayer('cartodbpositron',control=True, overlay=True, min_zoom=5, zoom_start=6.2).add_to(m)
icon = folium.features.CustomIcon(path2+'east.png',icon_size=(120,90))
folium.Marker(location=[40.2747,134.85077],icon=icon).add_to(m)

col_lst=['1인가구수', '실업률'] #''안에 확인하고 싶은 컬럼명들 넣기

for col in col_lst:
    bins = list(sigungu_data[col].quantile([0, 0.1, 0.35, 0.65, 0.9, 1]))

    folium.Choropleth(
        geo_data=sigungu_shp, #지역 폴리곤(시도(sido_shp),시군구(sigungu_shp))
        name=col,
        data=sigungu_data, #통계데이터(시도(sido_data),시군구(sigungu_data))
        columns=['시군구',col], #시도/시군구
        key_on='feature.properties.'+'SIG_KOR_NM', #시도(CTP_KOR_NM),시군구(SIG_KOR_NM)
        line_weight=0.1,
        fill_color='RdBu_r',#높은수치가 안전하면 RdBu, 낮은수치가 안전하면 RdBu_r, 
        fill_opacity=0.7,
        nan_fill_opacity=0,
        overlay=False,
        bins=bins,
        legend_name=col,
        highlight=True,
        reset=True
    ).add_to(m)

#tooltip만들기(마우스 올리면 위치에 따른 정보 보이는 것)
style_function = lambda x: {'fillColor': '#ffffff', #말풍선색(흰색)
                            'color':'#000000', #글자색(검정)
                            'fillOpacity': 0.1, 
                            'weight': 0.1}

ttip = folium.features.GeoJson(
    sigungu_shp,
    style_function=style_function,
    control=False, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['SIG_KOR_NM'], #시도(CTP_KOR_NM),시군구(SIG_KOR_NM)
        aliases=['시군구명'] #시도(시도명),시군구(시군구명)
    )
)

ttip.add_to(m)

m.keep_in_front(ttip)

folium.LayerControl(collapsed=False).add_to(m)


#지도가 안 보이면 아랫줄 주석 풀어 html로 저장해서 html문서 열어봐도 된다
#m.save('choropleth1.html')